In [1]:
%matplotlib inline
import os
import csv
import pandas as pd
import numpy as np

# import Well Location(geospatial info) csv files as Pandas dataframes

In [2]:
# Specify the columns you want to read
columns_to_read = ['WCRNUMBER', 'DECIMALLATITUDE', 'DECIMALLONGITUDE','BOTTOMOFPERFORATEDINTERVAL',\
            'TOTALCOMPLETEDDEPTH','TOPOFPERFORATEDINTERVAL','WELLYIELD','TESTTYPE','TOTALDRAWDOWN',\
            'PUMPTESTLENGTH','PUMPTESTLENGTH','STATICWATERLEVEL',]

try:
    # Read CSV file with specified columns
    df1 = pd.read_csv("wellcompletionreports.csv", usecols=columns_to_read)
    
    # Display the first few rows to verify
    print(df1.head())
except FileNotFoundError:
    print("File not found.")
except Exception as e:
    print(f"Error: {e}")

C:\Users\betebari\AppData\Local\Temp\ipykernel_5924\3123040521.py:8: DtypeWarning: Columns (14,15,32,33,34,41,42) have mixed types. Specify dtype option on import or set low_memory=False.
  df1 = pd.read_csv("wellcompletionreports.csv", usecols=columns_to_read)


        WCRNUMBER DECIMALLATITUDE DECIMALLONGITUDE TOTALCOMPLETEDDEPTH  \
0  WCR2005-016846         37.6724         -121.724                 400   
1  WCR2006-007057         37.5258          -122.03                  20   
2  WCR2006-000009             NaN              NaN                 NaN   
3  WCR2006-002311             NaN              NaN                 NaN   
4  WCR2006-002049             NaN              NaN                 250   

  TOPOFPERFORATEDINTERVAL BOTTOMOFPERFORATEDINTERVAL  STATICWATERLEVEL  \
0                     175                        355               NaN   
1                      10                         20              8.22   
2                     NaN                        NaN               NaN   
3                     NaN                        NaN               NaN   
4                     NaN                        NaN             19.00   

   TOTALDRAWDOWN TESTTYPE PUMPTESTLENGTH WELLYIELD  
0            NaN      NaN            NaN       NaN  
1   

## Convert '/' character to decimal character
# i.e.   37/41/13.06/  to 37+41  divide by /60  + 13.06 # divide by 3600

In [3]:
def convert_latitude(lat):
    if isinstance(lat, str) and '/' in lat:
        lat_parts = lat.split('/')[:-1]  # Exclude the last element
        try:
            degrees, minutes, seconds = map(float, lat_parts)
            decimal_lat = degrees + (minutes / 60) + (seconds / 3600)
            return decimal_lat
        except ValueError:
            return None
    else:
        try:
            return float(lat)
        except (ValueError, TypeError):
            return None

def convert_longitude(lon):
    if isinstance(lon, str) and '/' in lon:
        lon_parts = lon.split('/')[:-1]  # Exclude the last element
        try:
            degrees, minutes, seconds = map(float, lon_parts)
            decimal_lon = degrees + (minutes / 60) + (seconds / 3600)
            return decimal_lon
        except ValueError:
            return None
    else:
        try:
            return float(lon)
        except (ValueError, TypeError):
            return None

In [4]:
df1['Decimal_Lat'] = df1['DECIMALLATITUDE'].apply(convert_latitude)
df1['Decimal_Long'] = df1['DECIMALLONGITUDE'].apply(convert_longitude)

In [5]:
df1 = df1.dropna(subset=['DECIMALLONGITUDE','DECIMALLATITUDE','TOTALDRAWDOWN' ,'WELLYIELD'])
df1 = df1.drop(columns=['DECIMALLONGITUDE','DECIMALLATITUDE' ])
# print(df1.head())

In [6]:
# Convert columns to numeric, setting errors='coerce' to convert invalid parsing to NaN
df1['TOTALDRAWDOWN'] = pd.to_numeric(df1['TOTALDRAWDOWN'], errors='coerce')
df1['TOTALCOMPLETEDDEPTH'] = pd.to_numeric(df1['TOTALCOMPLETEDDEPTH'], errors='coerce')

# Create the 'drawdown_flag' column based on the condition
df1['drawdown_flag'] = (df1['TOTALDRAWDOWN'] > df1['TOTALCOMPLETEDDEPTH']) & \
                       (~df1['TOTALCOMPLETEDDEPTH'].isna()) & \
                       (df1['TOTALCOMPLETEDDEPTH'] != df1['TOTALDRAWDOWN'])

# Print 'not reliable' if any value in 'drawdown_flag' is True
if df1['drawdown_flag'].any():
    print("not reliable")

# Display the DataFrame
# print(df1.head())

not reliable


# Estimate of Transmisivity using Driscoll approach (198)

#Estimate of Transmisivity using Driscoll approach (1986, units gpd/ft, best if aquifer is Confined)
#Using equation (1), Driscoll (1986) assumed the following values to develop approximate formulas for estimating transmissivity from specific capacity in confined and unconfined aquifers:

#T = 30,000 US gal/day/ft

#S = 0.001 (confined) or 0.075 (unconfined)

#rw = 0.5 ft

#t = 1 day

#T = 2000(Q/sw) confined aquifer

#T = 1500(Q/sw) unconfined aquifer

#The reliability of specific capacity data from pumping tests depends on various factors, including the aquifer type, well construction, and testing conditions. In many typical scenarios, specific capacities less than 200 gpm/ft are considered realistic and reliable. Here is a more detailed breakdown of what is generally considered reliable for different aquifer types:
#1Unconsolidated Sand and Gravel Aquifers:** 

#Reliable specific capacities: 10 to 100 gpm/ft
#Higher values may occur but should be verified for accuracy.

#2.Sandstone Aquifers:**

#Reliable specific capacities: 1 to 20 gpm/ft
#Values higher than this range are rare and should be verified.

#3.Limestone or Karst Aquifers:**

#Reliable specific capacities: 5 to 50 gpm/ft

#Higher values can occur but are less common and may need careful verification.

#4.Fractured Rock Aquifers:**

#Reliable specific capacities: less than 5 gpm/ft
#Higher values are unusual and typically indicate specific geological features.

#Specific Capacity Ranges and Reliability

#Less than 1 gpm/ft:** Usually indicative of low-yield wells or poor aquifer conditions.

#1 to 10 gpm/ft:** Common in less productive aquifers such as fractured rocks or poorly sorted sands.

#10 to 100 gpm/ft:** Typical for many sand and gravel aquifers, as well as some limestone aquifers.

#100 to 200 gpm/ft:** Found in highly productive sand and gravel or karst aquifers but still within a reasonable range for certain geological settings.

#Summary

#For most practical purposes, specific capacities less than 200 gpm/ft are considered reasonable and reliable, particularly in unconsolidated sand and gravel aquifers, and some karst aquifers. Values above 200 gpm/ft should be carefully scrutinized to ensure accuracy and to understand the geological context.

In [7]:
# Convert columns to numeric, setting errors='coerce' to convert invalid parsing to NaN
df1['WELLYIELD'] = pd.to_numeric(df1['WELLYIELD'], errors='coerce')
df1['BOTTOMOFPERFORATEDINTERVAL'] = pd.to_numeric(df1['BOTTOMOFPERFORATEDINTERVAL'], errors='coerce')
df1['TOPOFPERFORATEDINTERVAL'] = pd.to_numeric(df1['TOPOFPERFORATEDINTERVAL'], errors='coerce')
df1['TOTALCOMPLETEDDEPTH'] = pd.to_numeric(df1['TOTALCOMPLETEDDEPTH'], errors='coerce')
df1['STATICWATERLEVEL'] = pd.to_numeric(df1['STATICWATERLEVEL'], errors='coerce')

#  calculate SpecificCapacity using:
df1['SpecificCapacity'] = df1['WELLYIELD'] / df1['TOTALDRAWDOWN']

# Set SpecificCapacity to NaN where it is greater than 200
df1.loc[df1['SpecificCapacity'] > 200, 'SpecificCapacity'] = pd.NA

#  Estimate of Transmisivity using Driscoll approach
df1['Tr_Driscoll_Confined'] = df1['SpecificCapacity'] *2000 
df1['Tr_Driscoll_Unconfined'] = df1['SpecificCapacity'] *1500 

# Round down the specified columns to the nearest 2 decimal places
df1['Tr_Driscoll_Confined'] = df1['Tr_Driscoll_Confined'].apply(lambda x: np.floor(x * 100) / 100)
df1['Tr_Driscoll_Unconfined'] = df1['Tr_Driscoll_Unconfined'].apply(lambda x: np.floor(x * 100) / 100)

# calculate Saturated thickness using top and bottom of screen
df1['SaturatedThickness'] = df1['TOTALCOMPLETEDDEPTH'] -df1['STATICWATERLEVEL'] 

# Replace negative values with NaN
df1['SaturatedThickness'] = np.where(df1['SaturatedThickness'] < 0, np.nan, df1['SaturatedThickness'])

# Where ScreenedInterval is NaN, calculate it using TOTALCOMPLETEDDEPTH and STATICWATERLEVEL
df1.loc[df1['SaturatedThickness'].isna(), 'SaturatedThickness'] = df1['BOTTOMOFPERFORATEDINTERVAL'] - df1['TOPOFPERFORATEDINTERVAL']

# Estimate of Hydraluic Conductivity using Driscoll approach
df1['Kxy_Driscoll_Confined'] = df1['Tr_Driscoll_Confined'] /df1['SaturatedThickness']
df1['Kxy_Driscoll_Unonfined'] = df1['Tr_Driscoll_Unconfined'] /df1['SaturatedThickness']

# Round down the specified columns to the nearest 2 decimal places
df1['Kxy_Driscoll_Confined'] = df1['Kxy_Driscoll_Confined'].apply(lambda x: np.floor(x * 100) / 100)
df1['Kxy_Driscoll_Unonfined'] = df1['Kxy_Driscoll_Unonfined'].apply(lambda x: np.floor(x * 100) / 100)


# Display the DataFrame
# print(df1.head())

# computing T in m2/day are (Batu 1998):

#If we express Q in m3/day and sw in m, the equations for computing T in m2/day are (Batu 1998):

#T = 1.385(Q/sw)    confined aquifer

#T = 1.042(Q/sw)    unconfined aquifer

In [8]:
# unit conversion 
df1['WELLYIELD_m3d'] = df1['WELLYIELD'] * 5.45099296896
df1['TOTALDRAWDOWN_m'] = df1['TOTALDRAWDOWN'] * 0.3048
df1['SaturatedThickness_m'] = df1['SaturatedThickness'] * 0.3048

df1['SpecificCapacity_m'] = df1['WELLYIELD_m3d'] / df1['TOTALDRAWDOWN_m']

df1['Tr_Batu_Confined'] = df1['SpecificCapacity_m'] * 1.385 
df1['Tr_Batu_Unconfined'] = df1['SpecificCapacity_m'] * 1.042 

# Round down the specified columns to the nearest 2 decimal places
df1['Tr_Batu_Confined'] = df1['Tr_Batu_Confined'].apply(lambda x: np.floor(x * 100) / 100)
df1['Tr_Batu_Unconfined'] = df1['Tr_Batu_Unconfined'].apply(lambda x: np.floor(x * 100) / 100)


# Estimate of Hydraluic Conductivity using Batu approach
df1['Kxy_Batu_Confined'] = df1['Tr_Batu_Confined'] /df1['SaturatedThickness_m']
df1['Kxy_Batu_Unonfined'] = df1['Tr_Batu_Unconfined'] /df1['SaturatedThickness_m']

# Round down the specified columns to the nearest 2 decimal places
df1['Kxy_Batu_Confined'] = df1['Kxy_Batu_Confined'].apply(lambda x: np.floor(x * 100) / 100)
df1['Kxy_Batu_Unonfined'] = df1['Kxy_Batu_Unonfined'].apply(lambda x: np.floor(x * 100) / 100)

# Display the DataFrame
# print(df1.head())

# Transmissivity in fractured bedrock  using Mace(1997) (m2/day)

In [9]:
df1['Tr_Mace_fracuture'] =0.76 * ((df1['SpecificCapacity_m']) ** 1.08)

# Estimate of Hydraluic Conductivity using mace approach
df1['Kxy_Mace_fracuture'] = df1['Tr_Mace_fracuture'] /df1['SaturatedThickness_m']

# Round down the specified columns to the nearest 2 decimal places
df1['Tr_Mace_fracuture'] = df1['Tr_Mace_fracuture'].apply(lambda x: np.floor(x * 100) / 100)
df1['Kxy_Mace_fracuture'] = df1['Kxy_Mace_fracuture'].apply(lambda x: np.floor(x * 100) / 100)

# Display the DataFrame
# print(df1.head())

# Transmisivity Razack & Huntley (m2/day)

In [10]:
df1['Tr_Razack_m2d'] =15.3 * ((df1['SpecificCapacity_m'] ** 0.67))
df1['Tr_Razack_ft2d'] =33.6 * (((192.5 * df1['SpecificCapacity'])) ** 0.67)

# Round down the specified columns to the nearest 2 decimal places
df1['Tr_Razack_m2d'] = df1['Tr_Razack_m2d'].apply(lambda x: np.floor(x * 100) / 100)
df1['Tr_Razack_ft2d'] = df1['Tr_Razack_ft2d'].apply(lambda x: np.floor(x * 100) / 100)

# Estimate of Hydraluic Conductivity using mace approach
df1['Kxy_Razack_md'] = df1['Tr_Razack_m2d'] /df1['SaturatedThickness_m']
df1['Kxy_Razack_ftd'] = df1['Tr_Razack_m2d'] /df1['SaturatedThickness']

# Round down the specified columns to the nearest 2 decimal places
df1['Kxy_Razack_md'] = df1['Kxy_Razack_md'].apply(lambda x: np.floor(x * 100) / 100)
df1['Kxy_Razack_ftd'] = df1['Kxy_Razack_ftd'].apply(lambda x: np.floor(x * 100) / 100)

# Display the DataFrame
print(df1)


              WCRNUMBER  TOTALCOMPLETEDDEPTH  TOPOFPERFORATEDINTERVAL  \
36       WCR2021-009337                500.0                    120.0   
176      WCR2020-008268                300.0                    160.0   
632      WCR1998-000135                370.0                    100.0   
831      WCR2020-009133                  NaN                      NaN   
879      WCR2022-003003                340.0                    200.0   
...                 ...                  ...                      ...   
1075388  WCR2004-000760                150.0                     38.0   
1075411  WCR2005-000437                450.0                    225.0   
1075423  WCR2005-003261                100.0                     32.0   
1075442  WCR2013-003719                400.0                      NaN   
1075561  WCR2001-002338                 81.0                     59.0   

         BOTTOMOFPERFORATEDINTERVAL  STATICWATERLEVEL  TOTALDRAWDOWN  \
36                            440.0             60.

In [11]:
# Save the updated DataFrame to a new CSV file
output_file = 'estimates of Transmissivity.csv'
df1.to_csv(output_file, index=False)

print(f"Transmissivity CSV file saved as '{output_file}'")

Transmissivity CSV file saved as 'estimates of Transmissivity.csv'
